<h1> Логико-вероятностный метод </h1>

<h2>Инициализация</h2>

Установка зависимостей

In [ ]:
!pip install -r requirements.txt

In [ ]:
import matplotlib.pyplot as plt
from pyvis.network import Network
# import webbrowser
import json

Инициализация глобальных переменных

In [ ]:
resource_name = input("Введите название ресурса")
graph_path = "F:\PycharmProjects\\UUSTDecisionTheory\graphs\\"
data_path = "F:\PycharmProjects\\UUSTDecisionTheory\data\\"

Функции для работы с сетью pyvis

In [ ]:
def add_event(graph):
    index = len(list(graph.nodes))
    event_name = input(f'Введите название события {index}')
    logit = input("Введите логит события (or - или, and - и, `вероятность` - инициирующее событие)")
    graph.add_node(index, label=f"{index}-{event_name}", title=logit)
    print(f"Событие '{event_name}' добавлено как '{logit}' под индексом {index}.\n")


def edit_event(graph):
    index = int(input(f'Введите индекс события'))
    event_name = input(f'Введите новое название события {index}')
    logit = input("Введите новый логит события (or - или, and - и, ini - инициирующее событие)")
    graph.delete_node(index)
    graph.add_node(index, label=f"{index}-{event_name}", title=logit)
    print(f"Событие '{event_name}' отредактировано как '{logit}' под индексом {index}.\n")


def delete_event(graph):
    event_index = int(input("Введите индекс события для удаления"))

    if event_index in list(graph.nodes):
        graph.remove_node(event_index)
        print(f"Событие '{event_index}' и все связанные с ним зависимости успешно удалены.\n")
    else:
        print(f"Событие '{event_index}' не найдено в графе.\n")


def add_dependency(graph):
    source_event_index = int(input("Введите индекс исходного события"))
    target_event_index = int(input("Введите индекс целевого события"))

    if (source_event_index in [node['id'] for node in graph.nodes] and
            target_event_index in [node['id'] for node in graph.nodes]):
        graph.add_edge(source_event_index, target_event_index)
        print(f"Зависимость от '{source_event_index}' к '{target_event_index}' добавлена.\n")
    else:
        print("Одно из событий не существует. Пожалуйста, добавьте их сначала.\n")


# FIXME
def make_levels(graph):
    node_positions = {}

    def assign_level(node, level=0):
        node_positions[node] = level
        for edge in graph.edges:
            if edge['from'] == node:
                assign_level(edge['to'], level + 1)

    assign_level(0)

    for node in graph.nodes:
        level = node_positions.get(node['id'], 0)
        graph.nodes[node['id']]['x'] = level * 200
        graph.nodes[node['id']]['y'] = -level * 100


def beautify(graph):
    graph.nodes[0]['color'] = 'red'
    graph.nodes[0]['size'] = 30

    # make_levels(graph)


def visualize_graph(graph):
    beautify(graph)

    file_absolute_path = graph_path + graph.nodes[0]['label'].split('-')[1] + ".html"
    graph.show(file_absolute_path)
    # FIXME
    # webbrowser.get('windows-default').open_new(file_absolute_path)
    print(f'Граф сохранен в файле {file_absolute_path}')


def save_graph_to_json(graph, filename):
    graph_data = {
        "nodes": [{"id": node["id"], "label": node["label"].split('-')[1], "title": node["title"]} for node in
                  graph.nodes],
        "edges": [{"from": edge["from"], "to": edge["to"]} for edge in graph.edges]
    }

    with open(filename, 'w') as f:
        json.dump(graph_data, f)
    print(f"Граф сохранен в файл {filename}")


def load_graph_from_json(filename):
    with open(filename, 'r') as f:
        graph_data = json.load(f)

    net = Network(notebook=True)

    for node in graph_data['nodes']:
        net.add_node(node['id'], label=f"{node['id']}-{node['label']}", title=node['title'])
        net.add_node(node['id'], label=node['label'], title=node['title'])

    for edge in graph_data['edges']:
        net.add_edge(edge['from'], edge['to'])

    return net

Инициализация графов

In [ ]:
critical_states_num = int(input("Введите количество опасных состояний ресурса"))


def validate_index(index):
    if index >= critical_states_num:
        raise ValueError("index out of range")


critical_states_graphs = [Network(directed=True, notebook=True, cdn_resources='local')] * critical_states_num

<h2>Работа с графом событий</h2>

Добавление событий

In [ ]:
state_index = int(input(f'Введите индекс состояния для добавления событий'))
validate_index(state_index)
G = critical_states_graphs[state_index]
add_event(G)

Редактирование событий

In [ ]:
state_index = int(input(f'Введите индекс состояния для редактирования событий'))
validate_index(state_index)
G = critical_states_graphs[state_index]
edit_event(G)

Удаление события

In [ ]:
state_index = int(input(f'Введите индекс состояния для удаление события'))
validate_index(state_index)
G = critical_states_graphs[state_index]
delete_event(G)

Добавление связей

In [ ]:
state_index = int(input(f'Введите индекс состояния для добавления зависимостей'))
validate_index(state_index)
G = critical_states_graphs[state_index]
add_dependency(G)

Визуализация

In [351]:
state_index = int(input(f'Введите индекс состояния для визуализации'))
validate_index(state_index)
G = critical_states_graphs[state_index]
visualize_graph(G)

F:\PycharmProjects\UUSTDecisionTheory\graphs\DB drop.html
Граф сохранен в файле F:\PycharmProjects\UUSTDecisionTheory\graphs\DB drop.html


Сохранение в файл

In [ ]:
state_index = int(input(f'Введите индекс состояния для сохранения'))
validate_index(state_index)
filename = str(input(f'Введите название файла для сохранения'))
save_graph_to_json(G, data_path + filename + ".json")

Считывание из файла

In [356]:
state_index = int(input(f'Введите индекс состояния для считывания'))
validate_index(state_index)
filename = str(input(f'Введите название файла для считывания'))
critical_states_graphs[state_index] = load_graph_from_json(data_path + filename + ".json")

<h2>Расчёт вероятностей реализации опасных состояний</h2>

Функции для расчёта

In [357]:
def get_prob(graph, node):
    edges_from = list(filter(lambda edge: edge['from'] == node['id'], graph.edges))
    logit = node['title']
    probability = None
    if logit == 'and':
        probability = 1
        for edge in edges_from:
            probability *= get_prob(graph, graph.nodes[edge['to']])
    elif logit == 'or':
        probability = 0
        for edge in edges_from:
            probability += get_prob(graph, graph.nodes[edge['to']])
    else:
        probability = float(node['title'])
    return probability

Расчёт вероятностей

In [358]:
probabilities = []
for i in range(critical_states_num):
    graph = critical_states_graphs[state_index]
    root = graph.nodes[0]
    probability = get_prob(graph, root)
    probabilities.append(probability)

Ввод потерь от реализации опасных состояний

In [359]:
losses = []
for i in range(critical_states_num):
    loss = float(input(f"Введите потери от реализации состояния {critical_states_graphs[i].nodes[0]['label'].split('-')[1]}"))
    losses.append(loss)

Расчёт риска реализации опасных состояний

In [361]:
risks = []
total = 0
for i in range(critical_states_num):
    risks.append(losses[i] * probabilities[i])
    print(f"Риск релизации состояния {critical_states_graphs[i].nodes[0]['label'].split('-')[1]}: {risks[i]}")
    total += risks[i]
print(f"Риск ИС: {total}")

Риск релизации состояния DB drop: 37650.0
Риск релизации состояния Lack of experienced specialists: 200800.0
Риск релизации состояния Server overload: 150600.0
Риск ИС: 389050.0
